In [1]:
from mp_api.client import MPRester
from mpkey import key
def fetch_magnetic_materials(api_key):
    m = MPRester(api_key)

    # Query the API for materials with a non-zero total magnetic moment
    
    materials = m.summary.search(
                        fields =["material_id","total_magnetization",'structure'])

    return materials

api_key = key  # Replace with your actual API key
materials = fetch_magnetic_materials(api_key)

/home/trial/miniconda3/envs/matgl-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Retrieving SummaryDoc documents: 100%|██████████| 154718/154718 [03:08<00:00, 820.41it/s] 


In [2]:
from pymatgen.io.vasp.inputs import Poscar
f = open("train-folder/id_prop.csv", "w")
for i in materials:
    atoms = i.structure
    
    jid = i.material_id.lstrip()
    target = i.total_magnetization
    if target < 0.05:
        target = -1.0
    else:
        target = 1.0
        
    poscar_name = "POSCAR-" + jid + ".vasp"
    poscar = Poscar(atoms)
    
    if target != "na":
        with open('train-folder/' + poscar_name, "w") as fil:
            fil.write(str(poscar))
            
        f.write("%s,%6f\n" % (poscar_name, target))

f.close()

No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


In [1]:
import json
# Define a configuration dictionary for the ALIGNN model
data = {
    "version": "112bbedebdaecf59fb18e11c929080fb2f358246",
    "dataset": "user_data",
    "target": "target",
    "atom_features": "cgcnn",
    "neighbor_strategy": "k-nearest",
    "id_tag": "jid",
    "random_seed": 123,
    "classification_threshold": 0.0,
    "n_val": None,
    "n_test": None,
    "n_train": None,
    "train_ratio": 0.8,
    "val_ratio": 0.1,
    "test_ratio": 0.1,
    "target_multiplication_factor": None,
    "epochs": 200,
    "batch_size": 32,
    "weight_decay": 1e-05,
    "learning_rate": 0.001,
    "filename": "sample",
    "warmup_steps": 2000,
    "criterion": "mse",
    "optimizer": "adamw",
    "scheduler": "onecycle",
    "pin_memory": False,
    "save_dataloader": False,
    "write_checkpoint": True,
    "write_predictions": True,
    "store_outputs": True,
    "progress": True,
    "log_tensorboard": False,
    "standard_scalar_and_pca": False,
    "use_canonize": True,
    "num_workers": 0,
    "cutoff": 8.0,
    "max_neighbors": 12,
    "n_early_stopping": 15,
    "keep_data_order": False,
    "n_early_stopping": 15,
    "model": {
        "name": "alignn",
        "alignn_layers": 4,
        "gcn_layers": 4,
        "atom_input_features": 92,
        "edge_input_features": 80,
        "triplet_input_features": 40,
        "embedding_features": 64,
        "hidden_features": 256,
        "output_features": 1,
        "link": "identity",
        "zero_inflated": False,
        "classification": True,
        "num_classes": 2
    }
}

with open('train-folder/config.json', 'w') as f:
    json.dump(data, f)

In [ ]:
#nohup train_folder.py --root_dir "train-folder" --config "train-folder/config.json" --output_dir "magmom-class" --classification_threshold 0.05 &